PyTorch's automatic registration happens only when assigning `nn.Module` (sub)class instances (it would be an error to do so for arbitrary member assignments). Otherwise we have to perform the registration manually.

In [1]:
from fastai.vision.all import *


We first import shape parameters.

In [2]:
pickle_path = URLs.path('mnist_png')/'mnist_png.pkl'
path = untar_data(URLs.MNIST)/'training'

if not pickle_path.exists():
    pickle_path.parent.mkdir(parents=True, exist_ok=True)
    ds = DataBlock(
        blocks = (ImageBlock(PILImageBW), CategoryBlock),
        get_items = get_image_files,
        get_y = parent_label,
        splitter = RandomSplitter(1/6, seed=0)
    ).datasets(path)

    xs, ys = zip(*ds.train, *ds.valid)
    xs = np.stack(L(map(lambda x: np.array(x, dtype=np.float32).reshape(-1), xs))) / 255.
    ys = np.array(ys, dtype=np.int64)

    x_train, x_valid = xs[:len(ds.train)], xs[len(ds.train):]
    y_train, y_valid = ys[:len(ds.train)], ys[len(ds.train):]

    save_pickle(pickle_path, [x_train, y_train, x_valid, y_valid])

    del ds, xs, ys, x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = map(tensor, load_pickle(pickle_path))



In [3]:
n, m = x_train.shape
c = y_train.max() + 1
nh = 50

bs = 50                # batch size

lr = 0.5   # learning rate
epochs = 3 # how many epochs to train for


Suppose we assign submodules in this manner. We then have to use `.add_module` to register the children.

In [4]:
from functools import reduce

layers = lambda: [nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10)]

class Model(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers
        for i, l in enumerate(self.layers):
            self.add_module(f'layer_{i}', l)

    def forward(self, x):
        return reduce(lambda val, layer: layer(val), self.layers, x)

In [5]:
model = Model(layers())
model


Model(
  (layer_0): Linear(in_features=784, out_features=50, bias=True)
  (layer_1): ReLU()
  (layer_2): Linear(in_features=50, out_features=10, bias=True)
)

In [6]:
xb, yb = x_train[:bs], y_train[:bs]
model(xb).shape


torch.Size([50, 10])

We have a helper method for this approach in `nn.ModuleList`.

In [7]:
class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def forward(self, x):
        for l in self.layers: x = l(x)
        return x


In [8]:
model = SequentialModel(layers())
model


SequentialModel(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [9]:
loss_func = F.cross_entropy
def accuracy(out, yb):
    return (out.argmax(dim=1) == yb).float().mean()
def report(loss, preds, yb):
    print(f'{loss:.2f}, {accuracy(preds, yb):.2f}')

def fit(model):
    for epoch in range(epochs):
        for i in range(0, n, bs):
            s = slice(i, min(n, i + bs))
            xb,yb = x_train[s], y_train[s]
            preds = model(xb)
            loss = loss_func(preds, yb)
            loss.backward()
            with torch.no_grad():
                for p in model.parameters(): p -= p.grad * lr
                model.zero_grad()
        report(loss, preds, yb)

fit(model)


0.23, 0.92
0.19, 0.96
0.15, 0.96


A similar helper class is `nn.Sequential`.

In [10]:
model = nn.Sequential(*layers())

fit(model)
loss_func(model(xb), yb), accuracy(model(xb), yb)


0.27, 0.94
0.24, 0.96
0.19, 0.96


(tensor(0.1552, grad_fn=<NllLossBackward0>), tensor(0.9400))